<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
from tqdm.notebook import tqdm
import warnings
import transformers
from tqdm import notebook
import re

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


from catboost import Pool, cv

from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
    cross_val_score
)

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
from nltk.stem import WordNetLemmatizer

import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
import spacy
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score, make_scorer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

RANDOM_STATE = 12345

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,9,alignment on this subject and which are contrary to those of DuLithgow,0


In [5]:
# Посмотри кол-во уникальных значений
display(data['toxic'].value_counts())
# Посмотрим соотношение в классах
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

Классы не сбалансированы

In [6]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = nlp(text)
  #  lemm_text = "".join(lemmatizer.lemmatize(text))
   # clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
  # clear_text = clear_text.split()
   # clear_text = " ".join(clear_text)
    return " ".join([token.lemma_ for token in lemm_text])

In [7]:

#sentence = "The striped bats are hanging on their feet for best"

#doc = nlp(sentence)

#" ".join([token.lemma_ for token in doc])



In [8]:
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lemmatize_text)

data = data.drop(['text'], axis=1)
data = data.drop(['Unnamed: 0'], axis=1)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [9]:
data.head()

,toxic,lemm_text
0,0,"explanation \n why the edit make under my username hardcore metallica fan be revert ? they be not vandalism , just closure on some gas after I vote at new york doll fac . and please do not remove the template from the talk page since I be retire now.89.205.38.27"
1,0,"d'aww ! he match this background colour I be seemingly stuck with . thank . ( talk ) 21:51 , january 11 , 2016 ( utc )"
2,0,"hey man , I be really not try to edit war . it be just that this guy be constantly remove relevant information and talk to I through edit instead of my talk page . he seem to care more about the formatting than the actual info ."
3,0,""" \n more \n I can not make any real suggestion on improvement - I wonder if the section statistic should be later on , or a subsection of "" "" type of accident "" "" -i think the reference may need tidy so that they be all in the exact same format ie date format etc . I can do that later on , if no - one else do first - if you have any preference for format style on reference or want to do it yourself please let I know . \n\n there appear to be a backlog on article for review so I guess there may be a delay until a reviewer turn up . it be list in the relevant form eg wikipedia : good_article_nominations#transport """
4,0,"you , sir , be my hero . any chance you remember what page that be on ?"


In [10]:
data_train, data_test = train_test_split(data,
                                         test_size=0.5, 
                                         random_state=RANDOM_STATE)

features_train = data_train.drop(['toxic'], axis=1)
target_train = data_train['toxic']

features_test = data_test.drop(['toxic'], axis=1)
target_test = data_test['toxic']

In [11]:
# считаем величину TF-IDF для обучающей выборки
count_tf_idf_train = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf_train.fit_transform(data_train.lemm_text)
display(tf_idf_train.shape)

(79646, 111895)

In [12]:
# считаем величину TF-IDF для тестовой выборки
count_tf_idf_test = TfidfVectorizer(stop_words=stopwords)
tf_idf_test = count_tf_idf_train.transform(data_test.lemm_text)
display(tf_idf_test.shape)

(79646, 111895)

## Обучение

In [13]:
model = LogisticRegression(random_state = RANDOM_STATE)

In [14]:
f1 = make_scorer(f1_score)

grid={"C": np.logspace(2.0, 4.0, num=5, base=2), "penalty":["l2"]} 

gsearch_LR = GridSearchCV(estimator=model, 
                       cv=5, 
                       param_grid=grid,
                       scoring = 'f1')

gsearch_LR.fit(tf_idf_train, target_train)    

KeyboardInterrupt: 

In [ ]:
print("Best Parameters:", gsearch_LR.best_params_)
print("Best Score:", gsearch_LR.best_score_)

In [15]:
classificator = DecisionTreeClassifier()
dict_classes={0:1, 1:class_ratio}
hyperparams = [{'max_depth':[x for x in range(50,100,2)],
                'random_state':[RANDOM_STATE],
                'class_weight':[dict_classes]}]

cv_counts = 2

clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(tf_idf_train, target_train)

DTC_best_params = clf.best_params_
print("Best Parameters:", DTC_best_params)
print()
print("Grid score:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

cv_f1_DTC = max(means)
print('F1 на cv', cv_f1_DTC)

Best Parameters: {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 90, 'random_state': 12345}

Grid score:

F1 на cv 0.6455898549305183


Делаем предсказания на тестовой выборке

In [ ]:
predictions = gsearch_LR.predict(tf_idf_test)
print('f1-score: {:.2f}'.format(f1_score(target_test, predictions)))

## Выводы

Были подготовлены данные, лемматизирован текст.

Данные разделены на обучающую и тестовую выборки.

Обучены две модели LR и DecisionTreeClassifier. С помощью LR получена модель со значением метрики качества F1 больше 0.75.
